# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate(压缩) them into dataloader form.

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)
print(len(train_dataset))
print(len(test_dataset))
# encapsulate them into dataloader form
# 可以多线程加速读取数据
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=10)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, num_workers=10)

In [ ]:
feature, label = train_dataset[0]
print(feature.shape, label)
# feature的尺寸是channel*height*width
# 其中channel是1，因为数据集是灰度图像

In [ ]:
start = time.time()
for X, y in train_loader:
    continue
print('%.2f sec' % (time.time() - start))

Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
# 网络的输入输出：784个像素，10个概率值
num_inputs = 784
num_outputs = 10

# LeNet
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(1, 6, 5), # in_channels, out_channels,kernel_size
                nn.Sigmoid(),
                nn.MaxPool2d(2, 2), # kernel_size, stride
                nn.Conv2d(6, 16, 5),
                nn.Sigmoid(),
                nn.MaxPool2d(2, 2)
                )
        self.fc = nn.Sequential(
                nn.Linear(16*4*4, 120),
                nn.Sigmoid(),
                nn.Linear(120, 84),
                nn.Sigmoid(),
                nn.Linear(84, 10)
                )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

model = SimpleNet()

# TODO:define loss function and optimiter
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in tqdm(data_iter):
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        labels_hat = model(images)
        loss = criterion(labels_hat, labels).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_l_sum += loss.item()
        train_acc_sum += (labels_hat.argmax(dim=1) == labels).sum().item()
        n += labels.shape[0]  
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    test_acc = evaluate_accuracy(test_loader, model)  
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
          % (epoch + 1, train_l_sum / n, train_acc_sum / n,
             test_acc))
    
print('time %.2f' % (time.time() - start_time))

#### Q5:
Please print the training and testing accuracy.

In [ ]:
print('loss %.4f, train acc %.3f, test acc %.3f'
          % (train_l_sum / n, train_acc_sum / n,
             test_acc))